## AAP-VLBI-Fit, automatic routine to fit VLBI intensity maps.

Please provide the files as follows: Each map (.fits/.IMAP) in its own folder, so that you have a list with the path+file for each map as well as another list with only the path where each map is located. We strongly suggest only fitting maps from the same source automatically. This is because the same detection limit may not necessarily be appropriate for each source.

In [ ]:
import glob
import importlib
import pandas as pd
from pathlib import Path
from astropy.io import fits ; from astropy.time import Time
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import main.main_functions as mf
importlib.reload(mf)

folders = pd.read_csv(f'../your_path/your_folders_list.txt',header=None) ;  folders = folders[0].tolist() ; display(folders)
files = pd.read_csv(f'../your_path/your_files_list.txt',header=None) ;  files = files[0].tolist() ; display(files)

Plase change the last line (327) in main_functions.py to "plt.close()", to avoid consuming too much memory in displaying the created figures. Variables that you need to change per source as denoted by an arrow and a comment.

In [ ]:
files_path,folders_path = files,folders

#Creation of the table with the summary of the fit
Properties = pd.DataFrame(columns=['File','Date','MJD','Peak_Flux','Total_Flux','Detection_Lim','Ratio','Components','Fit_Status','Clean_Status','Map_RMS','Model_RMS','Residual_RMS','Rate_Min','Rate_Max'])
print(f'Maps to be analized: {len(files_path)}')
for file,folder in zip(files_path,folders_path):
    print(f'File: {file}')
    file_name = Path(file).stem
    hdul_map = fits.open(file) ; hdr_map = hdul_map[0].header ; date = hdr_map['DATE-OBS'] ; print(f'Date: {date}')

    #Map Properties
    map_data,delta,increment,c_pix,f_peak,peak_id,MJD,BMaj,BMin,BPA,f_total = mf.map_properties(file)

    #Determination of the Detection Limit
    treshold = 8 #<---- Change how many times the RMS mean threshold will be taken as the detection limit everytime the source changed !!!
    det_limit = mf.detection_limit(treshold,file,map_data)

    #Fitting Map Components
    count,term = mf.map_fitting(folder,file,f_peak,peak_id,BMaj,BMin,BPA,det_limit)

    #Remotion of Bad Components
    no_comp,clean_term = mf.map_cleaning(folder,file,count,det_limit,BMaj,BMin,term)

    #RMS from results
    map_RMS,model_RMS,res_RMS,rate_min,rate_max = mf.RMS_estimations(folder,file,term)

    #Properties data export of the proccess 
    status = 'AutoFit' if term == 'Final' else 'ToCheck' ; ratio = (det_limit/f_peak)*100
    Properties.loc[len(Properties.index)] = [file_name,date,MJD,f_peak,f_total,det_limit,ratio,no_comp,status,clean_term,map_RMS,model_RMS,res_RMS,rate_min,rate_max]

    #Plotting and Component Summary Export
    mf.ploting(folder,file,term,det_limit,increment,f_peak,BMaj,BMin,BPA)
display(Properties)
Properties.to_csv(f'../your_path/Fit_Summary.txt',header=True,index=None,sep='\t') #<---- Change the saving route, if not it will overwrite in the same folder !!!